In [ ]:
import pyemu 

# We want to set up for global sensitivity analysis. 

In [ ]:
pst = pyemu.Pst('../noptmax0_testing/prior_mc_wide.pst')

### set `tie_by_group` to `True`. Also update the lower bound for CHD parameters to nearly 1.0 (because many of these values are at or near the bottom of model cells and if sampling sets them below the bottom of the cell, MODFLOW6 will not run). Also unfix the CHD parameters so they will be evaluated. All other defailts. for `pestpp-sen` will be accepted.

In [ ]:
pst.write('../run_data/prior_mc_wide_sens.pst')